### Read in and Examine Data

First, let's read in our data and make sure it looks right. We want to check there are no missing values and the data types are floats for latitude and longitude.

In [1]:
import pandas as pd

data = pd.read_csv('processed_headlines_locations.csv', index_col=0)
data[['headline', 'city', 'latitude', 'longitude', 'countrycode']].head()

,headline,city,latitude,longitude,countrycode
0,Zika Outbreak Hits Miami,Miami,25.77427,-80.19366,US
1,Could Zika Reach New York City?,New York City,40.71427,-74.00597,US
2,First Case of Zika in Miami Beach,Miami Beach,25.79065,-80.13005,US
3,"Mystery Virus Spreads in Recife, Brazil",Recife,-8.05389,-34.88111,BR
4,Dallas man comes down with case of Zika,Dallas,32.78306,-96.80667,US


In [2]:
print('Missing Values:')
data.isna().sum()

Missing Values:


headline         0
city             0
accented_city    0
countrycode      0
latitude         0
longitude        0
pop              0
dtype: int64

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 605 entries, 0 to 646
Data columns (total 7 columns):
headline         605 non-null object
city             605 non-null object
accented_city    605 non-null object
countrycode      605 non-null object
latitude         605 non-null float64
longitude        605 non-null float64
pop              605 non-null float64
dtypes: float64(3), object(4)
memory usage: 37.8+ KB


# Clustering using DBSCAN

In [5]:
from sklearn.cluster import DBSCAN

clusterer = DBSCAN()
clusterer

DBSCAN(algorithm='auto', eps=0.5, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=5, n_jobs=None, p=None)

### Distance Metric: Euclidean

In [6]:
def cluster_location(clusterer, data, lat_string="latitude", lon_string="longitude"):
    """
    Fit a clustering algorithm on location data.
    """
    features = data[[lat_string, lon_string]].copy()
    clusterer.fit(features)
    # Assign the cluster labels
    data["cluster"] = clusterer.labels_
    return data

In [7]:
data = cluster_location(clusterer, data)
data['cluster'].value_counts()

-1    476
 4     26
 1     21
 0     19
 6     14
 7     12
 3     12
 5      8
 9      6
 2      6
 8      5
Name: cluster, dtype: int64

In [8]:
clusterer.eps = 9
data = cluster_location(clusterer, data)
data['cluster'].value_counts()

 0     371
 2      59
 3      57
-1      32
 1      21
 5      16
 8       9
 4       9
 6       8
 11      7
 9       6
 7       6
 10      4
Name: cluster, dtype: int64

-1 in this library indicates the point is an outlier (noise) and has not been assigned to any cluster.

In [9]:
clusterer.min_samples = 3
data = cluster_location(clusterer, data)
data['cluster'].value_counts()

 0     373
 4      62
 2      59
 1      21
-1      16
 8      16
 11      9
 10      9
 5       9
 9       8
 12      6
 7       6
 3       5
 13      3
 6       3
Name: cluster, dtype: int64

In [10]:
best_euclidean_clusterer = DBSCAN(
    eps=9, min_samples=3, metric="euclidean",
)
data = cluster_location(best_euclidean_clusterer, data)

In [23]:
!pip install --user git+https://github.com/matplotlib/basemap.git

  Cloning https://github.com/matplotlib/basemap.git to /tmp/pip-req-build-xxkumzm4
  Running command git clone -q https://github.com/matplotlib/basemap.git /tmp/pip-req-build-xxkumzm4
  fatal: unable to access 'https://github.com/matplotlib/basemap.git/': The requested URL returned error: 504
ERROR: Command errored out with exit status 128: git clone -q https://github.com/matplotlib/basemap.git /tmp/pip-req-build-xxkumzm4 Check the logs for full command output.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [22]:
from matplotlib.basemap import Basemap
import matplotlib.pyplot as plt

plt.rcParams['font.size'] = 18

%matplotlib inline

# Simple world map
m = Basemap()
_ = m.shadedrelief()

ModuleNotFoundError: No module named 'matplotlib.basemap'

Drawing a simple map with `basemap` is very easy. Now, we have to map the latitude and longitude onto the map. This requires converting the latitude and longitude coordinates to the map coordinates.

In [ ]:
import matplotlib.pyplot as plt

# Make an empty figure
plt.figure(figsize=(16, 10))

# Convert the longitude, latitude to map projection coordinates
x, y = m(x=data["longitude"], y=data["latitude"])

m.shadedrelief()

# Plot the headline locations
plt.scatter(x, y, 20, marker="o", color="red")

# Grab the current axis to set the title (gca)
ax = plt.gca()
_ = ax.set_title("Locations of Headlines", size=20)

Plotting Clusters
Now, we need to associate colors with each cluster. The easiest way to do this is to iterate through each cluster and plot one cluster at a time with a different color. We'll add a legend for interpretation.

In [ ]:
plt.figure(figsize=(16, 10))

m.shadedrelief()
data["x"], data["y"] = x, y

# Iterate through each cluster and plot
for cluster, grouped in data.groupby("cluster"):
    plt.scatter(grouped["x"], grouped["y"], 20, marker="o", label=f"Cluster: {cluster}")

# Grab the figure to set the title and make a legend
ax = plt.gca()
ax.legend()
_ = ax.set_title("Locations of Headlines", size=20)

Now we have a map with each point colored by cluster membership. We can put our plotting code into a function to make our plots repeatedly. We'll color points not in any cluster black and let matplotlib choose colors for other clusters.

In [ ]:
def plot_clusters(data):
    """
    Plot clustered data on a basemap.
    """
    plt.figure(figsize=(16, 10))
    m.shadedrelief()

    # Iterate through each cluster and plot
    for cluster, grouped in data.groupby("cluster"):
        if cluster == -1:
            # Handle the unassigned headlines
            plt.scatter(
                grouped["x"],
                grouped["y"],
                s=60,
                alpha=0.8,
                marker="o",
                label=f"Cluster: None",
                c="k",
            )
        else:
            plt.scatter(
                grouped["x"],
                grouped["y"],
                s=60,
                alpha=0.8,
                marker="o",
                label=f"Cluster: {cluster}",
            )

    # Add a legend and title (put legend to right of plot)
    ax = plt.gca()
    ax.legend(loc=(1, 0))
    _ = ax.set_title("Locations of Headlines", size=20)

In [ ]:
plot_clusters(data)
plt.savefig('euclidean_clustering_map.png')

Implementation
To use the Haversine formula, we have to write a function which returns the Great Circle distance between two latitude and longitude numpy arrays

In [ ]:
import numpy as np

def great_circle_distance(coord1, coord2, radius=3956):
    """
    Calculates the great circle distance between two coordinates or arrays of coordinates.
    """
    if np.array_equal(coord1, coord2):
        return 0.0

    # Convert lat/lon to radians
    coord1, coord2 = np.radians(coord1), np.radians(coord2)
    # Find the difference between the coordinates
    delta_x, delta_y = coord2 - coord1
    
    # Apply Haversin formula
    haversin = np.sin(delta_x / 2) ** 2 + np.product(
        [np.cos(coord1[0]), np.cos(coord2[0]), np.sin(delta_y / 2) ** 2]
    )

    # Convert to distance in miles
    return 2 * radius * np.arcsin(haversin ** 0.5)

When we pass in coordinates, each column of the array should be a point. The first row contains the latitude and the second the longitude.

Let's test this on our first three headlines.

In [ ]:
data.head()

In [ ]:
coord1 = np.array(
    [
        [data["latitude"].iloc[0], data["latitude"].iloc[1]],
        [data["longitude"].iloc[0], data["longitude"].iloc[1]],
    ]
)

coord2 = np.array(
    [
        [data["latitude"].iloc[2], data["latitude"].iloc[0]],
        [data["longitude"].iloc[2], data["longitude"].iloc[0]],
    ]
)

# Calculate distance between points
great_circle_distance(coord1, coord2)

In [ ]:
from IPython.display import Image
Image('../figs/distance-check.png')

Using Great Circle Distance
To use the Great Circle Distance, we pass in the metric function (a callable) to the metric keyword parameter. We'll use a slightly different value - 250 miles - for eps now that we are using distance in miles between points. The min_samples will increase to 4 cities.

In [ ]:
great_circle_clusterer = DBSCAN(
    eps=250, min_samples=4, metric=great_circle_distance,
)

# Cluster using great circle distance
data = cluster_location(
     great_circle_clusterer, data, lon_string="longitude", lat_string="latitude"
)

data["cluster"].value_counts()

In [ ]:
Great Circle Clustering Map

In [ ]:
_ = plot_clusters(data)

In [ ]:
from IPython.display import Image
Image('../figs/euclidean_clustering_map.png')

Each clustering has mistakes, but overall, the Euclidean clustering appears better.

As one more experiment, let's trying using the Manhattan Distance between points for the metric. This simply adds together the x and y differences to get a total distance between points.

In [ ]:
manhattan_clusterer = DBSCAN(
    eps=9, min_samples=3, metric="manhattan",
)
data = cluster_location(manhattan_clusterer, data)

In [ ]:
_ = plot_clusters(data)

KMeans
The second method to try for clustering is the KMeans algorithm. KMeans places data points into k clusters - chosen ahead of time by the programmer - where each data point belongs to the cluster with the nearest mean. We'll use Scikit-Learn (sklearn.cluster.KMeans) for the implementation.

In [ ]:
from sklearn.cluster import KMeans

kmeans_clusterer = KMeans()
kmeans_clusterer

We'll try a range of values for the number of clusters, n_clusters. For each value, we'll record the inertia, the within-cluster sum-of-squares criterion. This is the loss KMeans attempts to minimize.

In [ ]:
inertia_values = []

for k in range(1, 11):
    inertia_values.append(KMeans(n_clusters=k).fit(data[['latitude', 'longitude']]).inertia_)

In [ ]:
plt.figure(figsize=(16, 10))
plt.plot(range(1, 11), inertia_values);
plt.title('Inertia Values by Number of Clusters');
plt.xlabel('Number of Clusters');
plt.ylabel('Intertia');

The elbow of the plot is at 3 clusters. At this value, we get a low inertia without a large number of clusters.

In [ ]:
kmeans_clusterer = KMeans(n_clusters=3)
data = cluster_location(kmeans_clusterer, data)
plot_clusters(data)

Three clusters appears to be too few. Let's increase it to 8.

In [ ]:
kmeans_clusterer = KMeans(n_clusters=8)
data = cluster_location(kmeans_clusterer, data)
plot_clusters(data)

World Plot
For the final output of this section, we'll create a plot using the best-identified clusterer without the outliers.

In [ ]:
best_clusterer = DBSCAN(eps=9, metric="euclidean", min_samples=3)
data = cluster_location(best_clusterer, data)
plot_clusters(data[data['cluster'] != -1])
plt.savefig('../figs/entire_world_clustering.png')

In [ ]:
plot_clusters(data[data['cluster'] != -1].sample(frac=0.10))
ax = plt.gca()
plt.rcParams['font.size'] = 22
ax.set_title('Sample of Headlines with Clusters');

Cluster Distribution
The clusters are not great - noticeably the entire United States is in one cluster. The distributions of points in clusters is also skewed, with one massive cluster and several smaller ones with only a few headlines

In [ ]:
_ = data['cluster'].value_counts().plot.bar(title='Cluster Distribution')

Nearly half of the headlines are from the United States, so we might have to cluster the data separately.

In [ ]:
data.groupby('countrycode')['cluster'].value_counts()['US']

Given the skewed distribution from the United States, it will make sense to cluster the US and the world separately. We'll do this in the next notebook. For now, we know how to use the algorithms and can see there are definitely groups of headlines which may indicate disease outbreaks.

In [ ]:
data.to_csv('../data/processed_headlines_clustered.csv')

In [ ]:
data[['headline', 'city', 'latitude', 'longitude', 'countrycode', 'cluster']].head(10)

In [ ]:
from IPython.display import Image

Image('../figs/entire_world_clustering.png')

In [ ]:
data[['headline', 'city', 'latitude', 'longitude', 'countrycode', 'cluster']].to_csv('../data/clustered_data.csv')